# Tensor Puzzles
- by [Sasha Rush](http://rush-nlp.com) - [srush_nlp](https://twitter.com/srush_nlp) 


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/srush/Tensor-Puzzles/blob/main/Tensor%20Puzzlers.ipynb)



When learning a tensor programming language like PyTorch or Numpy it
is tempting to rely on the standard library (or more honestly
StackOverflow) to find a magic function for everything.  But in
practice, the tensor language is extremely expressive, and you can
do most things from first principles and clever use of broadcasting.



This is a collection of 16 tensor puzzles. Like chess puzzles these are
not meant to simulate the complexity of a real program, but to practice
in a simplified environment. Each puzzle asks you to reimplement one
function in the NumPy standard library without magic. 


* [Rules](#Rules)
* [Puzzle 1 - ones](#puzzle-1---ones).
* [Puzzle 2 - sum](#puzzle-2---sum).
* [Puzzle 3 - outer](#puzzle-3---outer).
* [Puzzle 4 - diag](#puzzle-4---diag).
* [Puzzle 5 - eye](#puzzle-5---eye).
* [Puzzle 6 - triu](#puzzle-6---triu).
* [Puzzle 7 - cumsum](#puzzle-7---cumsum).
* [Puzzle 8 - diff](#puzzle-8---diff).
* [Puzzle 9 - vstack](#puzzle-9---vstack).
* [Puzzle 10 - roll](#puzzle-10---roll).
* [Puzzle 11 - flip](#puzzle-11---flip).
* [Puzzle 12 - compress](#puzzle-12---compress).
* [Puzzle 13 - pad_to](#puzzle-13---pad_to).
* [Puzzle 14 - sequence_mask](#puzzle-14---sequence_mask).
* [Puzzle 15 - bincount](#puzzle-15---bincount).
* [Puzzle 16 - scatter_add](#puzzle-16---scatter_add).

## Setup (<- Collapse this part!)

In [13]:
!pip install -qqq torchtyping hypothesis pytest notebookjs

In [14]:
# - 
gen_mode = True
save_mode = False
import notebookjs
import urllib
import torch
import time
rough_lib_url = "https://cdnjs.cloudflare.com/ajax/libs/rough.js/2.2.3/rough.es5.min.js"

js_string = """
function createHiPPICanvas(w, h) {
    let ratio = window.devicePixelRatio;
    let cv = document.createElement("canvas");
    cv.width = w * ratio;
    cv.height = h * ratio;
    cv.style.width = w + "px";
    cv.style.height = h + "px";
    cv.getContext("2d").scale(ratio, ratio);
    return {canvas: cv, ratio: ratio};
}

function draw_circle(div_id, data){
    name = data["name"];
    
    div = document.getElementById(div_id.substring(1));
    x = createHiPPICanvas(600, 500);
    var canvas = x.canvas;
    var ratio = x.ratio;
    var ctx = canvas.getContext("2d");
    ctx.font = "18px Georgia";

    ctx.fillText(name, 10, 20);
    ctx.font = "16px Georgia";
    canvas.setAttribute("style", "display: block;");
    div.appendChild(canvas);
    const rc = rough.canvas(canvas);

    m = 0;
    m_next = m;
    for (let ex = 0; ex < data["vals"].length; ex++) {
        vals = data["vals"][ex];
        var i = 0; 

        for (const k in vals) {
            y= draw_matrix(rc, ctx, k, vals[k], i * 150 + 50, 70 + m);
            if (y > m_next) {
                m_next = y;
            }
            i += 1;
        }
        m = m_next;
    }
    let comm = new CommAPI("save", () => {});
    comm.call({name:name, img:canvas.toDataURL()}); 
}

                    
function draw_matrix(rc, ctx, name, m, offx, offy) {
    off = 20;
    ctx.fillText(name, offx, offy-20);
    pad = 1
    for (let i = 0; i < m.length; i++){
       for (let j =0; j < m[0].length; j++){
        v = 'white';
        if (m[i][j] > 0) {
            v = 'rgb(255,100,0,' + (0.3 + (Math.abs(m[i][j]) / 6)) + ')';
        } else if (m[i][j] <0) {
            v = 'rgb(9,121, 105,' + (0.3 + (Math.abs(m[i][j]) / 6)) + ')';
        }
        
        opts = {stroke:'black', 
        roughness: 0.1,
        fillStyle:'cross-hatch', 
        fill: v}; // x, y, width, height
        
        if (m[i][j] == 0) {
            opts["fillStyle"] = "";
            opts["fill"] = "white";
        }
        rc.rectangle(offx + off * j, offy + off*i, off-pad, off-pad, opts);
       }
    }
    return m.length * off + offy
}
"""

# Code execution using notebookjs
from notebookjs import execute_js
def save(data):
    if save_mode:
        name, d = data["name"], data["img"]
        response = urllib.request.urlopen(d)
        with open("figs/" + name + '.png', 'wb') as f:
            f.write(response.file.read())
    else:
        None
    
def draw_example(data):
    execute_js([rough_lib_url, js_string], "draw_circle", data, {"save": save})
    
def draw_examples(name, examples):
    from IPython.display import Image, display
    if gen_mode:
        draw_example({"name":name,
          "vals" :[{k: [v.tolist()] if len(v.shape) ==1 else v.tolist() 
                        for k, v in example.items()}
                        for example in examples ] })
    else:
        display(Image(data=open("figs/" + name + ".png", "rb").read()))
        
from torchtyping import TensorType as TT
from hypothesis.extra.numpy import arrays
from hypothesis.strategies import integers, lists, composite, floats
from hypothesis import given
import numpy as np
import random
import sys
import typing
import matplotlib.pyplot as plt

tensor = torch.tensor

numpy_to_torch_dtype_dict = {
    bool: torch.bool,
    np.uint8: torch.uint8,
    np.int8: torch.int8,
    np.int16: torch.int16,
    np.int32: torch.int32,
    np.int64: torch.int64,
    np.float16: torch.float16,
    np.float32: torch.float32,
    np.float64: torch.float64,
}
torch_to_numpy_dtype_dict = {v: k for k, v in numpy_to_torch_dtype_dict.items()}


@composite
def spec(draw, x, min_size=1):
    # Get the type hints.
    if sys.version_info >= (3, 9):
        gth = typing.get_type_hints(x, include_extras=True)
    else:
        gth = typing.get_type_hints(x)

    # Collect all the dimension names.
    names = set()
    for k in gth:
        if not hasattr(gth[k], "__metadata__"):
            continue
        dims = gth[k].__metadata__[0]["details"][0].dims
        names.update([d.name for d in dims if isinstance(d.name, str)])
    names = list(names)

    # draw sizes for each dim.
    size = integers(min_value=min_size, max_value=5)
    arr = draw(arrays(shape=(len(names),), unique=True, elements=size, dtype=np.int32)).tolist()
    sizes = dict(zip(names, arr))

    # Create tensors for each size.
    ret = {}
    for k in gth:
        if not hasattr(gth[k], "__metadata__"):
            continue
        shape = tuple(
            [
                sizes[d.name] if isinstance(d.name, str) else d.size
                for d in gth[k].__metadata__[0]["details"][0].dims
            ]
        )
        dtype = (torch_to_numpy_dtype_dict[
                    gth[k].__metadata__[0]["details"][1].dtype
                ]
                if len(gth[k].__metadata__[0]["details"]) >= 2
                else int)
        ret[k] = draw(
            arrays(
                shape=shape,
                dtype=dtype,
                elements=integers(min_value=-5, max_value=5) if 
                         dtype == int else None,
                unique=False
            )
        )
        ret[k] = np.nan_to_num(ret[k], nan=0, neginf=0, posinf=0)

    ret["return"][:] = 0
    return ret, sizes


def make_test(name, problem, problem_spec, add_sizes=[], constraint=lambda d: d):
    examples = []
    for i in range(3):
        example, sizes = spec(problem, 3).example()
        example = constraint(example)
        out = example["return"].tolist()
        del example["return"]
        problem_spec(*example.values(), out)
        
        for size in add_sizes:
            example[size] = sizes[size]

        yours = None
        try:
            yours = problem(*map(tensor, example.values()))
            
        except AssertionError:
            pass
        for size in add_sizes:
            del example[size]
        example["target"] = tensor(out)
        if yours is not None:
            example["yours"] = yours 
        examples.append(example)
        
    draw_examples(name, examples)
    
    @given(spec(problem))
    def test_problem(d):
        d, sizes = d
        d = constraint(d)
        out = d["return"].tolist()
        del d["return"]
        problem_spec(*d.values(), out)
        for size in add_sizes:
            d[size] = sizes[size]

        out2 = problem(*map(tensor, d.values()))
        out = tensor(out)
        out2 = torch.broadcast_to(out2, out.shape)
        assert torch.equal(
            out, out2
        ), "Two tensors are not equal\n Spec: \n\t%s \n\t%s" % (out, out2)

    return test_problem


def run_test(fn):
    fn()
    # Generate a random puppy video if you are correct.
    print("Correct!")
    from IPython.display import HTML
    pups = [
    "2m78jPG",
    "pn1e9TO",
    "MQCIwzT",
    "udLK6FS",
    "ZNem5o3",
    "DS2IZ6K",
    "aydRUz8",
    "MVUdQYK",
    "kLvno0p",
    "wScLiVz",
    "Z0TII8i",
    "F1SChho",
    "9hRi2jN",
    "lvzRF3W",
    "fqHxOGI",
    "1xeUYme",
    "6tVqKyM",
    "CCxZ6Wr",
    "lMW0OPQ",
    "wHVpHVG",
    "Wj2PGRl",
    "HlaTE8H",
    "k5jALH0",
    "3V37Hqr",
    "Eq2uMTA",
    "Vy9JShx",
    "g9I2ZmK",
    "Nu4RH7f",
    "sWp0Dqd",
    "bRKfspn",
    "qawCMl5",
    "2F6j2B4",
    "fiJxCVA",
    "pCAIlxD",
    "zJx2skh",
    "2Gdl1u7",
    "aJJAY4c",
    "ros6RLC",
    "DKLBJh7",
    "eyxH0Wc",
    "rJEkEw4"]
    return HTML("""
    <video alt="test" controls autoplay=1>
        <source src="https://openpuppies.com/mp4/%s.mp4"  type="video/mp4">
    </video>
    """%(random.sample(pups, 1)[0]))


## Rules

1. Each puzzle needs to be solved in 1 line (<80 columns) of code.
2. You are allowed @, arithmetic, comparison, `shape`, any indexing (e.g. `a[:j], a[:, None], a[arange(10)]`), and previous puzzle functions.
3. Additionally you are allowed these two functions:

In [15]:
def arange(i: int):
    "Use this function to replace a for-loop."
    return torch.tensor(range(i))

draw_examples("arange", [{str(i) : arange(i)} for i in [5, 3, 9]])

In [16]:
def where(q, a, b):
    "Use this statement to replace an if-statement."
    return (q * a) + (~q) * b

# In my diagrams, orange is positive/True, where is zero/False, and green is negative.

examples = [(tensor([False]), tensor([10]), tensor([0])),
            (tensor([False, True]), tensor([1, 1]), tensor([-10, 0])),
            (tensor([False, True]), tensor([1]), tensor([-10, 0])),
            (tensor([[False, True], [True, False]]), tensor([1]), tensor([-10, 0])),
            (tensor([[False, True], [True, False]]), tensor([[0], [10]]), tensor([-10, 0])),
           ]
            

draw_examples("where", [{"q": q, "a":a, "b":b, "ret": where(q, a, b)} for q, a, b in examples])


### Anti-Rules

1. Nothing else. No `view`, `sum`, `take`, `squeeze`, `tensor`.
2. No cheating. Stackoverflow is great, but this is about first-principles.
3. Hint... these puzzles are mostly about Broadcasting. Make sure you understand this rule.

![](https://pbs.twimg.com/media/FQywor0WYAssn7Y?format=png&name=large)

In [17]:
examples = [(arange(4), arange(5)[:, None]) ,
            (arange(3)[:, None], arange(2))]
draw_examples("broadcast", [{"a": a, "b":b, "ret": a + b} for a, b in examples])


# Dims: a primer

Dims are python objects that represent a dimension in a tensor, or a in a set of tensors that share the same dimension. Dimensions are created with the `dims` function and bound to variables:


In [18]:
from dim import dims
from torch import where

In [19]:
i, j = dims()
k = dims()
print(i, j, k)

i j k


`dims` always returns the number of dimensions that are assigned to it. This is done using some C Python trickery, but is is the only "magical" thing about the API. It allows the dimensions to be created cheaply, and to provide the right names to the dim objects without redundancy.

Dims can be bound to particular size on creation, by setting their `.size` property later, or _binding_ the dimension to the dimension of an existing Tensor:

In [20]:
i.size = 3
x, y = dims(5, 6)
A = torch.randint(10, (4,))
a = A[j]

print(a)
print(a.ndim)

tensor([9, 1, 7, 6])
with dims=(j,) torch.Size([4])
0


Tensors like `a` which has a bound dimension `j` are implicitly batched over that dimension. Similiar to xmap, operations on tensors with bound dimensions will result in a tensor with the union of all of their dimensions so operations can be batched across multiple bound dimensions:

In [21]:
B = torch.randint(10, (3,)) * 100
b = B[i]
print(b)
print(a + b)

tensor([300, 200, 700])
with dims=(i,) torch.Size([3])
tensor([[309, 209, 709],
        [301, 201, 701],
        [307, 207, 707],
        [306, 206, 706]])
with dims=(j, i) torch.Size([4, 3])


Operations that normally take numbers to specify a dimension can how take a dimension object to act on that dimension:

In [22]:
print(a.sum(j))

tensor(23)


To 'unbind' a dimension, returning it back to a positional dimension, call the `positional` function, which will prepend that dimension to the positional dimensions of a tensor:

In [23]:
print(a.positional(j))

tensor([9, 1, 7, 6])


Tensors can have a mix of bound and unbound dimensions. When pretty printed, the `dims=` listing will indicate which physical dimensions of the tensor are bound to dimensions. Negative numbers indicate the positional index of unbound dimensions indexed from the right. It is printed this way because broadcast rules first right align the tensors, so the broadcasting of a tensor with mixed bound/unbound dimensions will be the union of the bound dimensions and negative integers in all the inputs.

In [24]:
C = torch.rand(3, 4)
D = torch.rand(4, 5)
print(C[i])
print(D[:, k])
print(C[i] + D[:, k])

tensor([[0.7208, 0.7437, 0.5185, 0.4025],
        [0.4226, 0.1594, 0.6546, 0.4325],
        [0.6184, 0.2549, 0.7148, 0.5443]])
with dims=(i, -1) torch.Size([3, 4])
tensor([[0.8196, 0.5994, 0.3172, 0.8566, 0.4267],
        [0.4182, 0.0527, 0.9666, 0.4099, 0.6620],
        [0.8302, 0.0130, 0.7828, 0.0457, 0.2299],
        [0.3119, 0.1769, 0.6141, 0.5387, 0.6078]])
with dims=(-1, k) torch.Size([4, 5])
tensor([[[1.5404, 1.3202, 1.0380, 1.5774, 1.1475],
         [1.1619, 0.7964, 1.7103, 1.1536, 1.4057],
         [1.3487, 0.5315, 1.3013, 0.5642, 0.7484],
         [0.7144, 0.5794, 1.0166, 0.9412, 1.0102]],

        [[1.2422, 1.0220, 0.7398, 1.2792, 0.8493],
         [0.5776, 0.2120, 1.1260, 0.5693, 0.8214],
         [1.4848, 0.6676, 1.4374, 0.7003, 0.8845],
         [0.7444, 0.6094, 1.0466, 0.9712, 1.0402]],

        [[1.4379, 1.2178, 0.9356, 1.4750, 1.0451],
         [0.6732, 0.3076, 1.2215, 0.6649, 0.9169],
         [1.5450, 0.7278, 1.4977, 0.7606, 0.9447],
         [0.8562, 0.7212, 1.1584,

The `A[i]` indexing which has been used to bind dimensions is just the advanced indexing operator for Torch's tensors extended to work with dimension objects. When a dimension object is directly used as an index, it binds the object to that dimension of the tensor, and sets the size of the dimension object to the Tensor's size. However, it is also possible to index a tensor with other tensors that have bound dimensions. This behaves just like any other operator that has bound dimensions. The output tensor will have a union of all the bound dimensions of the inputs. A very useful special case of this behavior is when the index tensors have no unbound dimensions. In this case, the indexing operation behaves the same as a similarly written loop nest but without doing the loop in Python:

In [25]:
A = torch.rand(5)
indices = torch.tensor([1, 3])
i = dims()

print(A[indices[i]])

print([A[indices[x]] for x in range(2)])

tensor([0.9071, 0.6241])
with dims=(i,) torch.Size([2])
[tensor(0.9071), tensor(0.6241)]


Dimension objects can be used as tensor values as well where they are the index along a dimension (concretely, they have the value `torch.arange(i.size)[i]`). This can be useful for computing masks:

In [60]:
i, j = dims(3, 3)
print(i.eq(j))
print(100*i + j)

tensor([[ True, False, False],
        [False,  True, False],
        [False, False,  True]])
with dims=(i, j) torch.Size([3, 3])
tensor([[  0,   1,   2],
        [100, 101, 102],
        [200, 201, 202]])
with dims=(i, j) torch.Size([3, 3])


Dims have more functionality: they can be used to split, merge, and reorder dimesions and `einsums` are automatically discovered by `sum` with `*`, but this information is sufficient to understand the answers to the puzzels below.

# Dim: New Rules
* Each solution can have 1 line to declare the dimensions, and 1 line to compute the result.
* pointwise arithmetic including where and numeric constants are allowed.
* You can use torch.tensor to lift a constant into a zero dim tensor, but not for any other use.
* @ (matrix multiply) is not allowed, but sum is allowed.
* arange is not allowed.

## Running puzzles

Each example, corresponds to a unit test which will randomly
try to break your code based on the spec. The spec is written in
standard python with lists.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/srush/Tensor-Puzzles/blob/main/Tensor%20Puzzlers.ipynb)

After you convince yourself it is right, uncomment the test for each example. If the test succeeds, you will get a puppy. 

## Puzzle 1 - ones

Compute [ones](https://numpy.org/doc/stable/reference/generated/numpy.ones.html) - the vector of all ones.

In [27]:
def ones_spec(out):
    for i in range(len(out)):
        out[i] = 1
        
def ones(i: int) -> TT["i"]:
    i = dims(int(i))
    return torch.tensor(1).expand(i).positional(i)

    # the `i` passed by the test framework is actually a tensor
    # so it has to be cast back to an int, normally you could just write dims(x) to create a dim
    # of size x.

    # One would rarely create
    # specifically sized tensors of all ones, instead
    # preferring to just use the constant 1 in arithmetic work with the constant 1


test_ones = make_test("one", ones, ones_spec, add_sizes=["i"])

In [28]:
run_test(test_ones)

Correct!


## Puzzle 2 - sum

Compute [sum](https://numpy.org/doc/stable/reference/generated/numpy.sum.html) - the sum of a vector.

In [29]:
def sum_spec(a, out):
    out[0] = 0
    for i in range(len(a)):
        out[0] += a[i]
        
def sum(a: TT["i"]) -> TT[1]:
    # sum makes more sense as a primitive than matrix multiply
    # so this puzzle no longer makes sense in this setup
    return a.sum()


test_sum = make_test("sum", sum, sum_spec)

In [30]:
run_test(test_sum)

Correct!


## Puzzle 3 - outer

Compute [outer](https://numpy.org/doc/stable/reference/generated/numpy.outer.html) - the outer product of two vectors.

In [31]:
def outer_spec(a, b, out):
    for i in range(len(out)):
        for j in range(len(out[0])):
            out[i][j] = a[i] * b[j]
            
def outer(a: TT["i"], b: TT["j"]) -> TT["i", "j"]:
    # the syntax closely matches the spec
    i, j = dims()
    return (a[i] * b[j]).positional(i, j)

    
test_outer = make_test("outer", outer, outer_spec)

In [32]:
run_test(test_outer)

Correct!


## Puzzle 4 - diag

Compute [diag](https://numpy.org/doc/stable/reference/generated/numpy.diag.html) - the diagonal vector of a square matrix.

In [33]:
def diag_spec(a, out):
    for i in range(len(a)):
        out[i] = a[i][i]
        
def diag(a: TT["i", "i"]) -> TT["i"]:
    # the syntax closely matches the spec
    i = dims()
    return a[i, i].positional(i)



test_diag = make_test("diag", diag, diag_spec)

In [34]:
run_test(test_diag)

Correct!


## Puzzle 5 - eye

Compute [eye](https://numpy.org/doc/stable/reference/generated/numpy.eye.html) - the identity matrix.

In [35]:
def eye_spec(out):
    for i in range(len(out)):
        out[i][i] = 1
        
def eye(j: int) -> TT["j", "j"]:
    # this is a functional definition of the spec
    i,j = dims(int(j), int(j))
    return where(i.eq(j), 1, 0).positional(i, j)

    
test_eye = make_test("eye", eye, eye_spec, add_sizes=["j"])

In [36]:
run_test(test_eye)

Correct!


## Puzzle 6 - triu

Compute [triu](https://numpy.org/doc/stable/reference/generated/numpy.triu.html) - the upper triangular matrix.

In [37]:
def triu_spec(out):
    for i in range(len(out)):
        for j in range(len(out)):
            if i <= j:
                out[i][j] = 1
            else:
                out[i][j] = 0
                
def triu(j: int) -> TT["j", "j"]:
    i,j = dims(int(j), int(j))
    return where(i <= j, 1, 0).positional(i, j)


test_triu = make_test("triu", triu, triu_spec, add_sizes=["j"])

In [38]:
run_test(test_triu)

Correct!


## Puzzle 7 - cumsum

Compute [cumsum](https://numpy.org/doc/stable/reference/generated/numpy.cumsum.html) - the cumulative sum.

In [39]:
def cumsum_spec(a, out):
    total = 0
    for i in range(len(out)):
        out[i] = total + a[i]
        total += a[i]

def cumsum(a: TT["i"]) -> TT["i"]:
    i,j = dims(a.size(0), a.size(0))
    return where(i <= j, a[i], 0).sum(i).positional(j)

test_cumsum = make_test("cumsum", cumsum, cumsum_spec)

In [40]:
run_test(test_cumsum)

Correct!


## Puzzle 8 - diff

Compute [diff](https://numpy.org/doc/stable/reference/generated/numpy.diff.html) - the running difference.

In [41]:
def diff_spec(a, out):
    out[0] = a[0]
    for i in range(1, len(out)):
        out[i] = a[i] - a[i - 1]

def diff(a: TT["i"], i: int) -> TT["i"]:
    i = dims()
    return where(input=(a[i] - a[i - 1]), other=a[i], condition=i - 1 >= 0).positional(i)
    # nit: this is calling where using keyword arguments so that i will bind
    # to the a.size(0) before we evalute the condition `i - 1 >= 0``
    # but it looks a bit odd

test_diff = make_test("diff", diff, diff_spec, add_sizes=["i"])

In [42]:
run_test(test_diff)

Correct!


## Puzzle 9 - vstack

Compute [vstack](https://numpy.org/doc/stable/reference/generated/numpy.vstack.html) - the matrix of two vectors

In [43]:
def vstack_spec(a, b, out):
    for i in range(len(out[0])):
        out[0][i] = a[i]
        out[1][i] = b[i]

def vstack(a: TT["i"], b: TT["i"]) -> TT[2, "i"]:
    v, i = dims(2)
    return where(v.eq(0),  a[i], b[i]).positional(v, i)


test_vstack = make_test("vstack", vstack, vstack_spec)

In [44]:
run_test(test_vstack)

Correct!


## Puzzle 10 - roll

Compute [roll](https://numpy.org/doc/stable/reference/generated/numpy.roll.html) - the vector shifted 1 circular position.

In [45]:
def roll_spec(a, out):
    for i in range(len(out)):
        if i + 1 < len(out):
            out[i] = a[i + 1]
        else:
            out[i] = a[i + 1 - len(out)]
            
def roll(a: TT["i"], i: int) -> TT["i"]:
    i = dims(a.size(0))
    return a[where(i + 1 < i.size, i + 1, 0)].positional(i)


test_roll = make_test("roll", roll, roll_spec, add_sizes=["i"])

In [46]:
run_test(test_roll)

Correct!


## Puzzle 11 - flip

Compute [flip](https://numpy.org/doc/stable/reference/generated/numpy.flip.html) - the reversed vector

In [47]:
def flip_spec(a, out):
    for i in range(len(out)):
        out[i] = a[len(out) - i - 1]
        
def flip(a: TT["i"], i: int) -> TT["i"]:
    i = dims(a.size(0))
    return a[i.size - i - 1].positional(i)


test_flip = make_test("flip", flip, flip_spec, add_sizes=["i"])

In [48]:
run_test(test_flip)

Correct!


## Puzzle 12 - compress


Compute [compress](https://numpy.org/doc/stable/reference/generated/numpy.compress.html) - keep only masked entries (left-aligned).

In [49]:
def compress_spec(g, v, out):
    j = 0
    for i in range(len(g)):
        if g[i]:
            out[j] = v[i]
            j += 1
            
def compress(g: TT["i", bool], v: TT["i"], i:int) -> TT["i"]:
    # this one is ugly (though I imagine the pytorch one has to be ugly as well)
    # it would be improved by being allowed to use setitem, or its functional equivalent
    i, j = dims(v.size(0), v.size(0))
    return where(where(g[j], where((j >= i) & g[i], 1, 0).sum(i) - 1, -1) == i, v[j], 0).sum(j).positional(i)



test_compress = make_test("compress", compress, compress_spec, add_sizes=["i"])

In [50]:
run_test(test_compress)

Correct!


## Puzzle 13 - pad_to


Compute pad_to - eliminate or add 0s to change size of vector.

In [51]:
def pad_to_spec(a, out):
    for i in range(min(len(out), len(a))):
        out[i] = a[i]


def pad_to(a: TT["i"], i: int, j: int) -> TT["j"]:
    k = dims(int(j))
    return where(k < i, a[where(k < i, k, 0)], 0).positional(k)
    # nit: it would be nice to have an indexing op that would result in an alternative tensor
    # where the original is not defined:
    # `a.orelse(0)[k]`, but I do not know any PyTorch equivalent to do this...


test_pad_to = make_test("pad_to", pad_to, pad_to_spec, add_sizes=["i", "j"])

In [52]:
run_test(test_pad_to)

Correct!


## Puzzle 14 - sequence_mask


Compute [sequence_mask](https://www.tensorflow.org/api_docs/python/tf/sequence_mask) - pad out to length per batch.

In [53]:
def sequence_mask_spec(values, length, out):
    for i in range(len(out)):
        for j in range(len(out[0])):
            if j < length[i]:
                out[i][j] = values[i][j]
            else:
                out[i][j] = 0
    
def sequence_mask(values: TT["i", "j"], length: TT["i", int]) -> TT["i", "j"]:
    j, i = dims()
    return where(input=values[i, j], other=0, condition=j < length[i]).positional(i, j)



def constraint_set_length(d):
    d["length"] = d["length"] % d["values"].shape[1]
    return d


test_sequence = make_test("sequence_mask",
    sequence_mask, sequence_mask_spec, constraint=constraint_set_length
)

In [54]:
run_test(test_sequence)

Correct!


## Puzzle 15 - bincount

Compute [bincount](https://numpy.org/doc/stable/reference/generated/numpy.bincount.html) - count number of times an entry was seen.

In [55]:
def bincount_spec(a, out):
    for i in range(len(a)):
        out[a[i]] += 1
        
def bincount(a: TT["i"], j: int) -> TT["j"]:
    j, i = dims(int(j))
    return where(a[i] == j, 1, 0).sum(i).positional(j)


def constraint_set_max(d):
    d["a"] = d["a"] % d["return"].shape[0]
    return d


test_bincount = make_test("bincount",
    bincount, bincount_spec, add_sizes=["j"], constraint=constraint_set_max
)

In [56]:
run_test(test_bincount)

Correct!


## Puzzle 16 - scatter_add

Compute [scatter_add](https://pytorch-scatter.readthedocs.io/en/1.3.0/functions/add.html) - add together values that link to the same location.

In [57]:
def scatter_add_spec(values, link, out):
    for j in range(len(values)):
        out[link[j]] += values[j]
        
def scatter_add(values: TT["i"], link: TT["i"], j: int) -> TT["j"]:
    j, i = dims(int(j))
    return where(link[i] == j, values[i], 0).sum(i).positional(j)


def constraint_set_max(d):
    d["link"] = d["link"] % d["return"].shape[0]
    return d


test_scatter_add = make_test("scatter_add",
    scatter_add, scatter_add_spec, add_sizes=["j"], constraint=constraint_set_max
)

In [58]:
run_test(test_scatter_add)

Correct!



# Speed Run Mode!

What is the smallest you can make each of these?

In [59]:
import inspect
fns = (ones, sum, outer, diag, eye, triu, cumsum, diff, vstack, roll, flip,
       compress, pad_to, sequence_mask, bincount, scatter_add)

for fn in fns:
    lines = [l for l in inspect.getsource(fn).split("\n") if not l.strip().startswith("#")]
    
    if len(lines) > 3:
        print(fn.__name__, len(lines[2]), "(more than 1 line)")
    else:
        print(fn.__name__, len(lines[1]))

ones 50 (more than 1 line)
sum 18
outer 41 (more than 1 line)
diag 32 (more than 1 line)
eye 48 (more than 1 line)
triu 47 (more than 1 line)
cumsum 54 (more than 1 line)
diff 89 (more than 1 line)
vstack 55 (more than 1 line)
roll 59 (more than 1 line)
flip 42 (more than 1 line)
compress 109 (more than 1 line)
pad_to 63 (more than 1 line)
sequence_mask 87 (more than 1 line)
bincount 54 (more than 1 line)
scatter_add 65 (more than 1 line)
